In [1]:
!pip install gradio


In [3]:
from google.colab import files
import zipfile

# Upload your zipped model folder
uploaded = files.upload()  # choose clip_finetuned.zip

# Unzip the model
with zipfile.ZipFile("clip_finetuned.zip", 'r') as zip_ref:
    zip_ref.extractall("clip_finetuned")

model_path = "clip_finetuned"  # folder with config.json & model.safetensors


Saving clip_finetuned.zip to clip_finetuned (1).zip


Load your fine-tuned model + tokenizer + precomputed embeddings

In [17]:
# import torch
# import pickle
# from PIL import Image
# from torchvision import transforms
# from transformers import CLIPModel, CLIPTokenizer
# import torch.nn.functional as F
# import os

# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load fine-tuned model

# # Check contents of the extracted directory
# extracted_files = os.listdir(model_path)
# print(f"Contents of {model_path}: {extracted_files}")

# # Assuming the actual model files are in a subfolder named 'Finetuned' based on zip content
# # Update model_path if necessary after inspecting the print output
# from transformers import CLIPProcessor

# # Point directly to the folder containing config.json + model.safetensors
# correct_model_path = "./clip_finetuned/Finetuned"

# try:
#     clip_model = CLIPModel.from_pretrained(correct_model_path).to(device)
#     clip_model.eval()
#     tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
#     processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

#     # Load embeddings
#     if device == "cpu":
#         image_embeddings = torch.load("image_embeddings.pt", map_location=torch.device('cpu'))
#     else:
#         image_embeddings = torch.load("image_embeddings.pt").to(device)

#     # Load image paths list
#     with open("images_list.pkl", "rb") as f:
#         image_paths = pickle.load(f)

#     # Image preprocessing (same as training)
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
#                              std=[0.26862954, 0.26130258, 0.27577711])
#     ])
# except Exception as e:
#     print(f"Error loading model: {e}")
#     print("Please check that config.json + model.safetensors exist in", correct_model_path)


Contents of clip_finetuned: ['Finetuned']


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [37]:
from transformers import CLIPModel, CLIPTokenizer, CLIPProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

# Point to your fine-tuned weights
model_path = "./clip_finetuned/Finetuned"

# Load fine-tuned model weights
clip_model = CLIPModel.from_pretrained(model_path).to(device)
clip_model.eval()

# Load tokenizer/processor from base pretrained model (not from your folder)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


Define caption → image retrieval function

In [32]:
from PIL import Image
import torch.nn.functional as F

def retrieve_image_by_caption(caption: str):
    # Tokenize caption
    tokens = tokenizer(caption, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        caption_embed = clip_model.get_text_features(
            input_ids=tokens.input_ids,
            attention_mask=tokens.attention_mask
        )
        caption_embed = F.normalize(caption_embed, dim=-1)

    # Similarity search
    similarities = torch.matmul(caption_embed, image_embeddings.T)
    top_idx = similarities.argmax().item()

    return images_list[top_idx]   # returns PIL.Image


Build Gradio UI

In [33]:
import gradio as gr
import torch

In [34]:
def gradio_search(caption):
    return retrieve_image_by_caption(caption)



In [35]:
with gr.Blocks() as demo:
    gr.Markdown("## 🖼️ CLIP Image Retrieval - Enter a caption")
    caption_input = gr.Textbox(label="Enter your caption")
    output_image = gr.Image(label="Retrieved Image")
    search_btn = gr.Button("Search")

    search_btn.click(fn=gradio_search, inputs=caption_input, outputs=output_image)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d61924cc10b712eb89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
